![Imagen](https://www.jumpdesign.co.uk/wp-content/uploads/2021/02/BANNER-LOGO-1024x288.jpg)
# **ETL - FIFA World Cup 2022**
### The objective of this Jupyter Notebook is generate a CSV that can feed a Power BI dashboard with all the data corresponding to every match in the FIFA World Cup 2022.

To achieve this we will use two different datasets:
1) The main dataset will be the following Kaggle dataset: [Main dataset](https://www.kaggle.com/datasets/die9origephit/fifa-world-cup-2022-complete-dataset)
2) The secondary dataset will be the following Kaggle dataset: [Secondary dataset](https://www.kaggle.com/datasets/swaptr/fifa-world-cup-2022-match-data)

We will take some features from the secondary dataset in order to have a more complete dataset and we will make the necessary transformations to have the dataset with a suitable format to feed the Power BI bashboard.

In [1]:
import pandas as pd
#pd.set_option("display.max_columns", None)
#pd.set_option("display.max_rows", None)

In [2]:
#Load CSVs
df1 = pd.read_csv("../FIFA_World_Cup_2022/FIFA_dataset/Fifa_world_cup_matches_v1.csv")
df2 = pd.read_csv("../FIFA_World_Cup_2022/FBref_matchs/data.csv")

In [3]:
#Normalization of some features in the data frame
df1["team1"] = df1["team1"].apply(lambda x: x.title())
df1["team2"] = df1["team2"].apply(lambda x: x.title())
df1.loc[df1["category"].str.contains("Group"), "category"] = "Group stage"
df1.loc[df1["category"] == "Play-off for third place", "category"] = "3rd place"

In [4]:
#Add additional information to the main data frame from the secondary data frame
additional_info = ["home_xg", "away_xg", "attendance", "venue", "referee", "home_formation", "away_formation", "home_touches", "away_touches", 
                "home_tackles", "away_tackles", "home_interceptions", "away_interceptions", "home_aerials_won", "away_aerials_won", 
                "home_clearances", "away_clearances", "home_gks", "away_gks", "home_throw_ins", "away_throw_ins", "home_long_balls", "away_long_balls"]
df1[additional_info] = df2[additional_info]
df1[["rival team1", "rival team2"]] = df1[["team2", "team1"]]

In [5]:
#Split team1 and team2 data
#Create team1 data frame
team1_columns = [col for col in df1.columns if "team2" not in col and "away" not in col]
team1 = df1[team1_columns]

In [6]:
#Create team2 data frame
team2_columns = [col for col in df1.columns if "team1" not in col and "home" not in col]
team2 = df1[team2_columns]
columns_team1 = team1.columns
columns_team2 = team2.columns
map_columns = dict(zip(columns_team2, columns_team1))
team2 = team2.rename(columns=map_columns)

In [7]:
#Concatenate both data frames
FWC2022 = pd.concat([team1, team2])

In [8]:
#Add team ID to data frame
all_teams = pd.read_csv("Teams.csv")
mapping = dict(zip(all_teams["team"], all_teams["team_id"]))
FWC2022["team_id"] = FWC2022["team1"].map(mapping)

In [9]:
#Reset and rename index
FWC2022 = FWC2022.reset_index(drop=True)
FWC2022 = FWC2022.rename_axis("match_id", axis=0)

In [10]:
#Manually replaced column names
FWC2022.rename(columns={'team1': "team", 'number of goals team1': "goals", 'category': "stage", 'venue': "stadium"}, inplace=True)
#Programmatically replaced column names
FWC2022.rename(columns=lambda x: x.replace(" team1", "").replace("home_", ""), inplace=True)

In [11]:
#Add the data corresponding to the rivals
top_col1 = FWC2022.iloc[:int(len(FWC2022)/2), 7:8]
top_col2 = FWC2022.iloc[:int(len(FWC2022)/2), 9:47]
top_col3 = FWC2022.iloc[:int(len(FWC2022)/2), 50:59]
top = pd.concat([top_col1, top_col2, top_col3], axis=1)
top.rename(columns=lambda x: x + "_against", inplace=True)

bottom_col1 = FWC2022.iloc[int(len(FWC2022)/2):, 7:8]
bottom_col2 = FWC2022.iloc[int(len(FWC2022)/2):, 9:47]
bottom_col3 = FWC2022.iloc[int(len(FWC2022)/2):, 50:59]
bottom = pd.concat([bottom_col1, bottom_col2, bottom_col3], axis=1)
bottom.rename(columns=lambda x: x + "_against", inplace=True)

against = pd.concat([bottom, top])
against = against.reset_index(drop=True)
FWC2022 = pd.concat([FWC2022, against], axis=1)

In [12]:
#Save the data frame
FWC2022.to_csv("WorldCup2022.csv")